In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 125

import seaborn as sns

import pandas as pd
import numpy as np
np.random.seed(0) # Giúp chạy các hàm của scikit-learn giống nhau mỗi lần chạy

import regex as re
import time # Dùng để sleep chương trình
from tqdm.notebook import tqdm # Hiện thanh progress cho đẹp :D
tqdm.pandas()

# Thư viện để request và parse HTML
import requests
from bs4 import BeautifulSoup

# Các thư viện liên quan tới ngôn ngữ và NLP
from pyvi import ViTokenizer # Thư viện NLP tiếng Việt
import gensim
import unicodedata # Thư viện unicode

# Trực quan hóa mô hình dự đoán văn bản
from lime import lime_text

# Dùng để lưu lại model
import pickle

# Thư viện liên quan của Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.preprocessing import LabelEncoder

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import feature_selection

# Tạo pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer, make_column_selector

# Các mô hình học
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier # Phương pháp bagging

In [2]:
# Thiết lập đường dẫn cho phần 1
dir_1 = "C:/Users/davin/Desktop/KHDL/"
# Thiết lập lại đường dẫn phần 1
dir_1_new = "C:/Users/davin/Desktop/KHDL/raw_data/"
# Thiết lập đường dẫn cho phần 2
dir_2 = "C:/Users/davin/Desktop/KHDL/pre_data/"

In [3]:
# KHÁM PHÁ VÀ TIỀN XỬ LÝ DỮ LIỆU BƯỚC ĐẦU
# Import data
df = []
for i in tqdm(range(40)):
    df.append(pd.read_csv(dir_1_new + f'crawling_{i}.csv'))
    
data_df = pd.concat(df)
data_df.reset_index(drop=True, inplace=True)
data_df.head()

  0%|          | 0/40 [00:00<?, ?it/s]

,links,title,description,class,content
0,https://tuoitre.vn/dien-bien-moi-thao-go-vuong...,"Diễn biến mới tháo gỡ vướng mắc các dự án, đất...",Chủ tịch Đà Nẵng là tổ trưởng tổ công tác của ...,Thời sự,UBND thành phố Đà Nẵng vừa có quyết định thành...
1,https://tuoitre.vn/xe-tai-xe-ben-chay-am-am-su...,"Xe tải, xe ben chạy ầm ầm suốt ngày đêm, sinh ...","Xe tải, xe ben phóng nhanh, bóp còi inh ỏi làm...",Bạn đọc làm báo,"Theo ghi nhận của Tuổi Trẻ Online, những chiếc..."
2,https://tuoitre.vn/ba-nguyen-thi-hong-hanh-duo...,Bà Nguyễn Thị Hồng Hạnh được bầu làm ủy viên U...,Giám đốc Sở Tư pháp TP.HCM Nguyễn Thị Hồng Hạn...,Thời sự,"Sáng 14-11, tại kỳ họp thứ 19, HĐND TP.HCM khó..."
3,https://tuoitre.vn/tong-thong-indonesia-tuyen-...,Tổng thống Subianto tuyên bố Indonesia sẽ bảo ...,Tân Tổng thống Indonesia Prabowo Subianto tuyê...,Thế giới,"Theo Hãng tin Reuters, bình luận nói trên của ..."
4,https://tuoitre.vn/ba-melania-trump-se-lam-de-...,Bà Melania Trump sẽ làm đệ nhất phu nhân bán t...,Một nguồn tin thân cận với bà Melania Trump - ...,Thế giới,Bà Melania Trump có kế hoạch chia quỹ thời gia...


In [4]:
# Kích thước của dữ liệu
data_df.shape

(160001, 5)

In [5]:
# kiểu dữ liệu
data_df.dtypes

links          object
title          object
description    object
class          object
content        object
dtype: object

In [6]:
# Kiểm tra dữ liệu trùng
data_df[data_df.duplicated(keep=False)].sort_values(by=['links'])

,links,title,description,class,content
43864,https://tuoitre.vn/10-du-an-dien-gio-nghin-ti-...,"10 dự án điện gió nghìn tỉ kêu khó mặt bằng, c...",10 dự án điện gió có tổng vốn đầu tư hơn 10.00...,Kinh doanh,"Sáng 27-3, UBND tỉnh Quảng Trị tổ chức cuộc họ..."
44267,https://tuoitre.vn/10-du-an-dien-gio-nghin-ti-...,"10 dự án điện gió nghìn tỉ kêu khó mặt bằng, c...",10 dự án điện gió có tổng vốn đầu tư hơn 10.00...,Kinh doanh,"Sáng 27-3, UBND tỉnh Quảng Trị tổ chức cuộc họ..."
131542,https://tuoitre.vn/10-nguoi-chet-11-nguoi-bi-t...,"10 người chết, 11 người bị thương vì tai nạn g...",TTO - Văn phòng Ủy ban An toàn giao thông quốc...,Thời sự,Số liệu trên được thống kê từ báo cáo của Cục ...
131529,https://tuoitre.vn/10-nguoi-chet-11-nguoi-bi-t...,"10 người chết, 11 người bị thương vì tai nạn g...",TTO - Văn phòng Ủy ban An toàn giao thông quốc...,Thời sự,Số liệu trên được thống kê từ báo cáo của Cục ...
129987,https://tuoitre.vn/10-o-to-ban-chay-nhat-thang...,"10 ô tô bán chạy nhất tháng 8: 7 xe gầm cao, n...",Xpander tiếp tục duy trì vị trí số 1 thị trườn...,Xe,NaN
...,...,...,...,...,...
25221,https://tuoitre.vn/xu-phat-nhieu-tiem-vang-o-c...,Xử phạt nhiều tiệm vàng ở Cần Thơ không niêm y...,"Các tiệm vàng ở Cần Thơ không niêm yết giá, ni...",Kinh doanh,"Ngày 4-7, Cục Quản lý thị trường TP Cần Thơ ch..."
111980,https://tuoitre.vn/xuan-truong-sa-ve-voi-truon...,Về với Trường Sa mùa xuân,Chương trình nghệ thuật đặc biệt ‘Xuân Trường ...,Văn hóa,Xuân Trường Sa diễn ra tối 8-1 tại Hà Nội với ...
112030,https://tuoitre.vn/xuan-truong-sa-ve-voi-truon...,Về với Trường Sa mùa xuân,Chương trình nghệ thuật đặc biệt ‘Xuân Trường ...,Văn hóa,Xuân Trường Sa diễn ra tối 8-1 tại Hà Nội với ...
129343,https://tuoitre.vn/yeu-to-khien-thi-truong-can...,Yếu tố khiến thị trường căn hộ Hà Nội giữ nhiệt,"Nguồn cung khan hiếm, căn hộ chung cư trở lại ...",Nhà đất,Đây được cho là 2 yếu tố then chốt khiến thị t...


In [7]:
# Kiểm tra dữ liệu thiếu
data_df.isna().sum()

links             0
title             0
description      13
class             0
content        5890
dtype: int64

In [8]:
# Hàm chuẩn hoá unicode
def covert_unicode(txt):
    if (type(txt) is not str):
        return txt
    return unicodedata.normalize('NFC', txt)

In [9]:
bang_nguyen_am = [['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
                  ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
                  ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
                  ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e'],
                  ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
                  ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i'],
                  ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o'],
                  ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
                  ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
                  ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u'],
                  ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
                  ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']]
bang_ky_tu_dau = ['', 'f', 's', 'r', 'x', 'j']

nguyen_am_to_ids = {}

for i in range(len(bang_nguyen_am)):
    for j in range(len(bang_nguyen_am[i]) - 1):
        nguyen_am_to_ids[bang_nguyen_am[i][j]] = (i, j)

def chuan_hoa_dau_tu_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word

    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    
    # Tách dấu ra khỏi từ và lưu lại vị trí nguyên âm
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1: # Phụ âm thì bỏ qua
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
                
        if y != 0: # Nếu có dấu
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
        if not qu_or_gi or index != 1:
            nguyen_am_index.append(index)
            
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            # Trường hợp chữ "gì"
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1])
                chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word

    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = bang_nguyen_am[x][dau_cau]
            return ''.join(chars)

    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
        else:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
    return ''.join(chars)


def is_valid_vietnam_word(word):
    # Từ hợp lệ là từ không có phụ âm bị kẹp giữa nguyên âm
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, _ = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if (index - nguyen_am_index) != 1:
                    return False
                nguyen_am_index = index
    return True


def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
    Chuyển câu tiếng việt về chuẩn gõ dấu kiểu cũ.
    :param sentence:
    :return:
    """
    if (type(sentence) is not str):
        return sentence
    
    words = sentence.split()
    for index, word in enumerate(words):
        # Tách chữ có dính dấu ngăn cách câu để chuẩn hóa
        cw = re.sub(r'(^\p{P}*)([\p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        # print(cw)
        # Chuẩn hóa nếu chữ không có vấn đề (thường fail khi chưa tách thành công các từ)
        if len(cw) == 3:
            cw[1] = chuan_hoa_dau_tu_tieng_viet(cw[1])
            words[index] = ''.join(cw)
    return ' '.join(words)

In [12]:
# Hàm tiền xử lý dữ liệu cho bước khám phá cuối cùng
def batch_preprocess():
    total_files = 40
    text_attrs = ["title","description","content","class"]
    
    for index in tqdm(range(total_files)):
        df=pd.read_csv(dir_1_new + f'crawling_{index}.csv')
        # Xóa dòng thiếu
        df=df[~((df['title'].isnull()) | (df['description'].isnull()) | (df['content'].isnull()))]
        for attr in text_attrs:
            df[attr]=df[attr].apply(covert_unicode)
            df[attr]=df[attr].str.lower()
            df[attr]=df[attr].apply(chuan_hoa_dau_cau_tieng_viet)
        df.to_csv(dir_2 + f'crawling_{index}.csv', index=False)

batch_preprocess()

  0%|          | 0/40 [00:00<?, ?it/s]